In [1]:
import pandas as pd
import numpy as np
import os

In [27]:
## Final Code Block to run

# Func to get key 
def get_key(indexes):
  op=["OPERATING", "OPERATIONS", "OPERATIONAL"]
  loss=["INCOME", "LOSS", "PROFIT", "(LOSS)", "(INCOME)"]

  temp=0
  for i in indexes:
    i=str(i)
    j=i.split()
    l1=[k for k in j if k in op]
    l2=[k for k in j if k in loss]
    if len(l1)!=0 and len(l2)!=0:
      temp=" ".join(j)
      break
  
  return temp



# Find synonyms of "OPERATING LOSS", as different companies use different names for "OPERATING LOSS".
# ebitda_list=["OPERATING LOSS", "OPERATING LOSS FROM CONTINUING OPERATIONS", "INCOME (LOSS) FROM OPERATIONS", "INCOME FROM OPERATIONS", "OPERATING INCOME (LOSS), TOTAL"]

## Assuming scale to be 'normal' initially for excel sheet of all years.


ebitda_change=pd.DataFrame(columns=["2021-2020", "2020-2019", "2019-2018", "2018-2017", "2017-2016"])
# ebitda_change=pd.DataFrame()

path="/content/drive/MyDrive/InterIIT/Data"
company_list=os.listdir(path)

for i in range(len(company_list)):
  print(company_list[i])
# for i in [-3,-1,5,11]:


  scale1="normal"
  scale2="normal"
  scale3="normal"

  sheets="/content/drive/MyDrive/InterIIT/Data/"+company_list[i]+"/10-K"

  try:
    lst=os.listdir(sheets)
  except Exception as e:
    print(e, f"related to company {company_list[i]}")
    ebitda_change.loc[company_list[i]]=np.nan
    continue
  
 
  # If a company has no data i.e no 10-K files are present.
  if len(lst)==0:
    ebitda_change.loc[company_list[i]]=np.nan
    continue
  
  # Sorting the list according to the modified time of file, which actually sorts list in descending order of years.
  lst.sort(key=lambda x: os.path.getmtime(sheets+"/"+x))

  print(lst)

  sheet1=pd.read_excel("/content/drive/MyDrive/InterIIT/Data/"+company_list[i]+"/10-K/"+lst[0], sheet_name=3, index_col=0)
  sheet1.index=sheet1.index.str.upper()

  drop_cols=[]
  for col_name in sheet1.iloc[0].isnull().index.values:
    if sheet1.iloc[0].isnull()[col_name]:
      drop_cols.append(col_name)
  sheet1.drop(drop_cols, axis=1, inplace=True)



  index_name1=sheet1.index.name.split()
  if "Thousands" in index_name1 or "thousands" in index_name1:
    scale1="thousands"
  elif "Millions" in index_name1 or "millions" in index_name1:
    scale1="millions"



  temp1=get_key(sheet1.index.values)
  if temp1==0:
    print(f"No key found for Ebitda for company {company_list[i]}")
    continue


  years=len(sheet1.loc[temp1])

  if years==2 and len(lst)>=3:

    sheet2=pd.read_excel("/content/drive/MyDrive/InterIIT/Data/"+company_list[i]+"/10-K/"+lst[1], sheet_name=3, index_col=0)
    sheet2.index=sheet2.index.str.upper()

    drop_cols=[]
    for col_name in sheet2.iloc[0].isnull().index.values:
      if sheet2.iloc[0].isnull()[col_name]:
        drop_cols.append(col_name)
    sheet2.drop(drop_cols, axis=1, inplace=True)

    
    index_name2=sheet2.index.name.split()
    if "Thousands" in index_name2 or "thousands" in index_name2:
      scale2="thousands"
    elif "millions" in index_name2 or "Millions" in index_name2:
      scale2="millions"


    temp2=get_key(sheet2.index.values)
    if temp2==0:
      print(f"No key found for Ebitda for company {company_list[i]}")
      continue

    sheet3=pd.read_excel("/content/drive/MyDrive/InterIIT/Data/"+company_list[i]+"/10-K/"+lst[2], sheet_name=3, index_col=0)
    sheet3.index=sheet3.index.str.upper()

    drop_cols=[]
    for col_name in sheet3.iloc[0].isnull().index.values:
      if sheet3.iloc[0].isnull()[col_name]:
        drop_cols.append(col_name)
    sheet3.drop(drop_cols, axis=1, inplace=True)


    index_name3=sheet3.index.name.split()
    if "Thousands" in index_name3 or "thousands" in index_name3:
      scale3="thousands"
    elif "millions" in index_name3 or "Millions" in index_name3:
      scale3="millions"


    temp3=get_key(sheet3.index.values)
    if temp3==0:
      print(f"No key found for Ebitda for company {company_list[i]}")
      continue


    col3=sheet3.iloc[0].values[0].split()[-1]+"-"+sheet3.iloc[0].values[1].split()[-1]
    ebitda_change.loc[company_list[i], col3]=(int(sheet3.loc[temp3][0])-int(sheet3.loc[temp3][1]))/100

    col2=sheet2.iloc[0].values[0].split()[-1]+"-"+sheet2.iloc[0].values[1].split()[-1]
    ebitda_change.loc[company_list[i], col2]=(int(sheet2.loc[temp2][0])-int(sheet2.loc[temp2][1]))/100

    col1=sheet1.iloc[0].values[0].split()[-1]+"-"+sheet1.iloc[0].values[1].split()[-1]
    ebitda_change.loc[company_list[i], col1]=(int(sheet1.loc[temp1][0])-int(sheet1.loc[temp1][1]))/100



    if scale1=="thousands":
      ebitda_change.loc[company_list[i], col1]=ebitda_change.loc[company_list[i], col1]*1000
    if scale2=="thousands":
      ebitda_change.loc[company_list[i], col2]=ebitda_change.loc[company_list[i], col2]*1000
    if scale3=="thousands":
      ebitda_change.loc[company_list[i], col3]=ebitda_change.loc[company_list[i], col3]*1000

    if scale1=="millions":
      ebitda_change.loc[company_list[i], col1]=ebitda_change.loc[company_list[i], col1]*1000000
    if scale2=="millions":
      ebitda_change.loc[company_list[i], col2]=ebitda_change.loc[company_list[i], col2]*1000000
    if scale3=="millions":
      ebitda_change.loc[company_list[i], col3]=ebitda_change.loc[company_list[i], col3]*1000000
      

  elif years==2 and len(lst)<3:
    print(company_list[i]+" doesn't has enough data")
    # If we dont have enough data, then we need to figure out that for how many years we can get ebitda_change.



  if years==3 and len(lst)>=2:
    sheet2=pd.read_excel("/content/drive/MyDrive/InterIIT/Data/"+company_list[i]+"/10-K/"+lst[1], sheet_name=3, index_col=0)
    sheet2.index=sheet2.index.str.upper()


    drop_cols=[]
    for col_name in sheet2.iloc[0].isnull().index.values:
      if sheet2.iloc[0].isnull()[col_name]:
        drop_cols.append(col_name)
    sheet2.drop(drop_cols, axis=1, inplace=True)


    temp2=get_key(sheet2.index.values)
    if temp2==0:
      print(f"No key found for Ebitda for company {company_list[i]}")
      continue
    

    index_name2=sheet2.index.name.split()
    if "Thousands" in index_name2 or "thousands" in index_name2:
      scale2="thousands"
    elif "millions" in index_name2 or "Millions" in index_name2:
      scale2="millions"

    
    col2=sheet2.iloc[0].values[1].split()[-1]+"-"+sheet2.iloc[0].values[2].split()[-1]
    ebitda_change.loc[company_list[i], col2]=(int(sheet2.loc[temp2][1])-int(sheet2.loc[temp2][2]))/100

    col1=sheet1.iloc[0].values[1].split()[-1]+"-"+sheet1.iloc[0].values[2].split()[-1]
    ebitda_change.loc[company_list[i], col1]=(int(sheet1.loc[temp1][1])-int(sheet1.loc[temp1][2]))/100

    col=sheet1.iloc[0].values[0].split()[-1]+"-"+sheet1.iloc[0].values[1].split()[-1]
    ebitda_change.loc[company_list[i], col]=(int(sheet1.loc[temp1][0])-int(sheet1.loc[temp1][1]))/100


    if scale2=="thousands":
      ebitda_change.loc[company_list[i], col2]=ebitda_change.loc[company_list[i], col2]*1000
    if scale1=="thousands":
      ebitda_change.loc[company_list[i], col1]=ebitda_change.loc[company_list[i], col1]*1000
      ebitda_change.loc[company_list[i], col]=ebitda_change.loc[company_list[i], col]*1000

    if scale2=="millions":
      ebitda_change.loc[company_list[i], col2]=ebitda_change.loc[company_list[i], col2]*1000000
    if scale1=="millions":
      ebitda_change.loc[company_list[i], col1]=ebitda_change.loc[company_list[i], col1]*1000000
      ebitda_change.loc[company_list[i], col]=ebitda_change.loc[company_list[i], col]*1000000



  elif years==3 and len(lst)<2:
    print(company_list[i]+" doesn't has enough data")
    # If we dont have enough data, then we need to figure out that for how many years we can get ebitda_change.

  

ebitda_change




PEGASYSTEMS INC
['000101385721000032.xlsx', '000101385720000007.xlsx', '000101385719000005.xlsx']
PING IDENTITY HOLDING CORP
['000155837021001638.xlsx', '000155837020002034.xlsx']
PROOFPOINT INC
['000156459021006943.xlsx', '000156459020005557.xlsx', '000156459019003699.xlsx', '000121245818000001.xlsx']
PROS HOLDINGS INC
['000139297221000039.xlsx', '000139297220000059.xlsx', '000139297219000067.xlsx', '000139297218000068.xlsx']
PTC INC
['000156459021057806.xlsx', '000156459020054803.xlsx', '000085700519000040.xlsx', '000085700518000025.xlsx']
Q2 HOLDINGS INC
['000141038421000022.xlsx', '000141038420000025.xlsx', '000141038419000015.xlsx', '000141038418000012.xlsx']
QAD INC
['000143774921008761.xlsx', '000143774920007629.xlsx', '000143774919007282.xlsx', '000143774918006885.xlsx']
QUALYS INC
['000143774921003648.xlsx', '000162828020001963.xlsx', '000162828019002099.xlsx', '000110784318000003.xlsx']
QUMU CORP
['000089248221000008.xlsx', '000089248220000007.xlsx', '000089248219000013.xlsx'

,2021-2020,2020-2019,2019-2018,2018-2017,2017-2016,2018-2019
PEGASYSTEMS INC,NaN,-86490.0,-1178460.0,-1102090.0,NaN,NaN
PING IDENTITY HOLDING CORP,NaN,-340180.0,-85290.0,-83940.0,NaN,NaN
PROOFPOINT INC,NaN,97350.0,-32200.0,-467820.0,NaN,NaN
PROS HOLDINGS INC,NaN,-127420.0,-41230.0,157280.0,NaN,NaN
PTC INC,1698850.0,1478210.0,-95710.0,NaN,NaN,NaN
Q2 HOLDINGS INC,NaN,-330870.0,-348960.0,-49430.0,NaN,NaN
QAD INC,183860.0,-152610.0,126470.0,NaN,NaN,NaN
QUALYS INC,NaN,244010.0,218920.0,131180.0,NaN,NaN
QUMU CORP,NaN,-19940.0,16030.0,19580000.0,NaN,NaN
RAPID7 INC,NaN,-281040.0,70430.0,-42440.0,NaN,NaN
